In [99]:
import folium
from pyproj import Proj, transform

In [108]:
# SACO DATOS

import requests

inicio = 'http://mapas.igme.es/gis/rest/services/Cartografia_Geologica/IGME_MAGNA_50/MapServer/10/query?'
f ='f=json' #Formato en el que pasara los datos
where = '&where=(1%3D1)%20AND%20(1%3D1)' #condicion para la consulta
geometry = '&returnGeometry=true'#si pasa la geometria o no
spatial = '&spatialRel=esriSpatialRelIntersects' #AVERIGUAR
outfields = '&outFields=*' #AVERIGUAR
orderby = '&orderByFields=OBJECTID%20ASC' #como viene ordenada la consulta
outSR = '&outSR=102100' #AVERIGUAR
resultOffset = '&resultOffset=167163' #cuantas columnas desde el principio antes de empezar a imprimir
resultRecord = '&resultRecordCount=100' #cuantas columnas imprimir

url = inicio+f+where+geometry+spatial+outfields+orderby+outSR+resultOffset+resultRecord
    
res = requests.get(url)
data = res.json()
item = data['features']
#item

In [109]:
# SEPARO LA LISTA DE RINGS

#rings = item[0]['geometry']
#target = rings['rings']
r = 0
target = []

#print(len(item))

while r < len(item):
    rings = item[r]['geometry']
    target.insert(r,rings['rings'])
    r += 1
#target

In [110]:
# RECORRO LA LISTA Y TRANSFORMO EL FORMATO DE LAS COORDENADAS
inProj = Proj(init='epsg:3857')
outProj = Proj(init='epsg:4326')
#lista = target[0]
resultado = []
i = 0
while i < (len(target)):
    lista = target[i]
    a = 0
    #print(lista)
    convertidas = []
    #print (len(lista[0]))
    while a < (len(lista[0])):
        #print(lista[0][a])
        x1,y1 = lista[0][a]
        punto = transform(inProj,outProj,x1,y1)
        convertidas.append(punto)
        a += 1
   # print(convertidas)
    resultado.insert(i,convertidas)
    i += 1
#resultado[4]

In [111]:
#PRUEBA DE INVERTIR TUPLAS

a = 0
final = []
while a < len(resultado):
    lista = resultado[a]
    #print(lista)
    b = 0
    item = []
    while b < len(lista):
        punto1 = lista[b][1]
        punto2 = lista[b][0]
        entrar = (punto1, punto2)
        item.append(entrar)
        b += 1
    #print(item)
    final.append(item)
    a += 1

#final[2]

In [112]:
#PRUEBA DE DIBUJO
m = folium.Map(location = final[0][0], zoom_start=11)

colors = [
    'red',
    'blue',
    'gray',
    'darkred',
    'lightred',
    'orange',
    'beige',
    'green',
    'darkgreen',
    'lightgreen',
    'darkblue',
    'lightblue',
    'purple',
    'darkpurple',
    'pink',
    'cadetblue',
    'lightgray',
    'black'
]

colint = 0
c = 0
while c < len(final):
    if colint == 18:
        colint = 0
    folium.Polygon(final[c], color=colors[colint], weight=2.5, fill_color= colors[colint], opacity=10, popup = 'Arcillas y limos. Zona endorreica').add_to(m)
    c += 1
    colint += 1
    
m